`cat urls.txt | while read f; do curl "${f}" -O; done;`

In [19]:
import time
import requests
import json
import subprocess
from bz2 import decompress
from datetime import datetime

In [2]:
urls = []
with open('../replays/urls.txt', 'r') as fin:
    for line in fin.readlines():
        urls.append(line.replace('\n', ''))
print(len(urls))
print(urls[:3])

54
['http://replay191.valve.net/570/6216665747_89886887.dem.bz2', 'http://replay192.valve.net/570/6215346651_640559417.dem.bz2', 'http://replay191.valve.net/570/6216545156_471262816.dem.bz2']


In [3]:
def download(url):
    r = requests.get(url)
    r.raise_for_status()
    compressed_dem = r.content
    dem = decompress(compressed_dem)
    return dem

In [4]:
def download_save(url):
    right = url.split('/')[-1]
    match_salt = right.replace('dem.bz2', '')
    file_name = match_salt.split('_')[0]
    file_name += '.dem'
    
    dem = download(url)
    path = f'../replays/{file_name}'
    with open(path, 'wb') as fout:
        fout.write(dem)
    return path

In [5]:
def download_parse_save(url):
    dem_path = download_save(url)
    jsonlines_path = dem_path.replace('.dem', '.jsonlines')
    cmd = f'curl localhost:5600 --data-binary "@{dem_path}" > {jsonlines_path}'
    subprocess.run(cmd, shell=True)
    return jsonlines_path

In [22]:
for url in urls:
    started_at = time.time()
    print(url, str(datetime.now())[:19])
    parsed_path = download_parse_save(url)
    print(parsed_path, time.time() - started_at)
    break

http://replay191.valve.net/570/6216665747_89886887.dem.bz2 2022-06-20 13:50:39


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 89  156M    0 24.5M   87  115M  7328k  33.6M  0:00:03  0:00:03 --:--:-- 40.8M

../replays/6216665747.jsonlines 14.041280269622803


100  163M    0 31.5M  100  132M  7545k  30.8M  0:00:04  0:00:04 --:--:-- 38.1M
